In [1]:
# use run all command before starting !

<div>
  <h1>Introduction and Background:</h1>
        <p>
            Football is one of the biggest sports of the modern and past ages. I would like to determine if Premier League teams from
            the past or present are better. I will be comparing the top 3 teams from 2019/20 season to the top 3 teams from 2006/07. 
            The reason for the specific chosen seasons are because the 2019/20 season was the last season that was played before covid-19 
            , which had a massive impact on the footballing world , and the 2006/07 season is the first season that the Premier League has
            collected statistics extensively. 
        </p>
        <p>The teams from 2006/07 are: Chelsea , Manchester United and Liverpool .  
            The teams from 2019/20 are: Chelsea , Manchester City and Liverpool .
        </p>
        <p> I aim to compare each team's statistics to conclude who was the best and why. I will also be comparing 3 teams of each season and also determine how football has changed from past to present.
            so that we can compare the opponents of the best teams in a given season. I will not be comparing players and their respective 
            stats. 
        </p>
</div>

<div>
    <p>
        This topic is very relevent since pundits ,supporters and players can't decide which teams are better. Although 
        many of these mentioned groups have strong opinions and beliefs , this field has not been analysed in a statistical comparison 
        and is subject to bias opinions more often then not. 
    </p>
</div>

<div>
    <h3>
        Evaluation method:
    </h3>
    <p>
        I will be drawing the statistics from each team , compressing it in a single dataframe and then start comparing columns 
        and rows of the dataframe to draw percentage and numerical statistics to compare and conclude my evaluation decision.
    </p>
</div>


<div>
    <h3>
    Data location
</h3>
    <p>
        The dataset I will be using i acquired from : <a href="https://www.premierleague.com">https://www.premierleague.com</a>
        I will be using a dataset on different pages for each team's stat page eg. <a href="https://www.premierleague.com/clubs/10/club/stats">https://www.premierleague.com/clubs/10/club/stats</a> 
        I will also be filtering the dataset on this page so a more accurate link eg. <a href="https://www.premierleague.com/clubs/10/club/stats?se=274">https://www.premierleague.com/clubs/10/club/stats?se=274</a> 
    </p>

</div>

<div>
    <h3>
        The following libraries I will be importing to use to work with my dataset. 
    </h3>
    <p>
        <strong>requests</strong> - To send HTTP request and use get method to get the text and data from the source.
    </p>
    <p>
        <strong>json</strong> - To get the filtered data from a XHR-reuest. 
    </p>
    <p>
        <strong>BeautifulSoup</strong> - To pull data out of html page.
    </p>
    <p>
        <strong>numpy</strong> - Performing post data extraction operations.
    </p>
    <p>
        <strong>csv</strong> - Accessing methods to write and read csv files.
    </p>
    <p>
        <strong>pandas</strong> - Data analysis and drawing visual tables.
    </p>
</div>

In [2]:
import requests
import json
from bs4 import BeautifulSoup
import numpy as np
import csv
import pandas as pd

<div>
      <h3>
        1) Data collection
    </h3>
    <p>
        I've written a function to collect the team statistics of each team called getTeamStats. This function takes 4 parameters:
    </p>
    <p>
        baseURL = The base URL of a given team eg. <a href="https://www.premierleague.com/clubs/11/club/stats">https://www.premierleague.com/clubs/11/club/stats</a>  
    </p>
    <p>
        dataURL = This URL is found by using the inspect tool on the above mentioned URL and navigating to the network tab and 
        using the "Fetch/XHR" filter and findiing the stats page. I did not know at first how to access this but I posted a 
        question on stackOverflow and was helped by a user : 
        <a href="https://stackoverflow.com/questions/70482520/web-scraping-after-a-filter-has-been-applied-to-data">https://stackoverflow.com/questions/70482520/web-scraping-after-a-filter-has-been-applied-to-data</a>
        I had to to this because you don't get filtered data (when apllying filter of the season year eg. 2019/20)
        because this data is loaded by Javascript using XHR-request. But you can send this request directly and get all needed data in JSON format.
        Therefore i had to import json to access the data I want. 
    </p>       
    <p>
        csvWriteFile = The csv file that the data is going to be written to. 
    </p>
    <p>
        teamSeason = One of the below defined integer values that will change the season of the data returned , this will be used
        to select the option that selects the desired filtered option of the page.
        eg. <a href="https://www.premierleague.com/clubs/10/club/stats?se=274">https://www.premierleague.com/clubs/10/club/stats?se=274</a>    The 'se=274' is for the season 2019/20.
    </p> 
    <p>
        This function returns the csv file's name for using later on.
    </p>
</div>

In [3]:
season_06 = 15
season_19 = 274

In [4]:
def getTeamStats(baseURL , dataURL , csvWriteFile,teamSeason):
    
    #using beautiful soup librarty to load text of baseURL
    TeamReq  = requests.get(baseURL)
    TeamData = TeamReq.text
    soup = BeautifulSoup(TeamData, "html.parser")

    headers = {
        'origin': 'https://www.premierleague.com',  # your get 403 Forbidden without this header
    }
    params = {
        "comps": 1,
        "compSeasons": teamSeason  # number of season to be used
    }
    
    # using get request to load text data of dataURL
    season_data = requests.get(dataURL, params=params, headers=headers)
    data = json.loads(season_data.text)

    StatNames = np.array([]) 
    StatVals = np.array([])
    team_db = {}

    # looping through all html tags of type span containing class 'allStatContainer' (baseURL)
    for stattext in soup.find_all("span",class_ ="allStatContainer"):
    
        #looping trough all stats in data (dataURL)
        for stat in data['stats']:
            
            # logic checking if the data-stat of an element is equal to stat name
            if stattext.get('data-stat')==stat['name']:
                
                # append different arrays with names and values simultaeously
                StatNames = np.append(StatNames,stat['name'])
                StatVals = np.append(StatVals,stat['value'])

    # Join the 2 arrays to create a dictionary of key value pairs
    for A,B in zip(StatNames,StatVals):
        team_db[A] = B
        
    # open csv file and write key value pairs as rows     
    with open(csvWriteFile,'w') as f:
        w = csv.writer(f)
        for key, value in team_db.items():
            w.writerow([key, value])
     
    
        

In [5]:
# Calling the function on each teams respective baseURL and data URL with a different csv filename and season

getTeamStats("https://www.premierleague.com/clubs/12/club/stats",
             "https://footballapi.pulselive.com/football/stats/team/12" ,
             "ManUtd_2006.csv" ,season_06)
getTeamStats("https://www.premierleague.com/clubs/4/club/stats",
             "https://footballapi.pulselive.com/football/stats/team/4" ,
             "Chelsea_2006.csv" ,season_06)
getTeamStats("https://www.premierleague.com/clubs/10/club/stats",
             "https://footballapi.pulselive.com/football/stats/team/10" ,
             "Liverpool_2006.csv" ,season_06 )
getTeamStats("https://www.premierleague.com/clubs/10/club/stats",
             "https://footballapi.pulselive.com/football/stats/team/10" ,
             "Liverpool_2019.csv" ,season_19 )
getTeamStats("https://www.premierleague.com/clubs/11/club/stats",
             "https://footballapi.pulselive.com/football/stats/team/11" ,
             "ManCity_2019.csv" , season_19)
getTeamStats("https://www.premierleague.com/clubs/4/club/stats",
             "https://footballapi.pulselive.com/football/stats/team/4" ,
             "Chelsea_2019.csv" , season_19)

<div>
     <h3>
        2) Data manipulation and Operations
    </h3>
    <p>
        Now there should be a csv file for each team containing their respective stats with key value pairs.
    </p>
    <p>
        Below I am assigning 'dataframe' values of each csv file that was created . Using the <strong>read_csv</strong>  method
        with the <strong>pandas </strong> library :
    </p>
</div>

In [6]:
df_ManUtd_06 = pd.read_csv('ManUtd_2006.csv',header=None )  # manUtd 2006
df_Chels_06 = pd.read_csv('Chelsea_2006.csv',header=None)   # Chelsea 2006
df_Liv_06 = pd.read_csv('Liverpool_2006.csv',header=None)   # Liverpool 2006
df_Liv_19 = pd.read_csv('Liverpool_2019.csv',header=None)   # Liverpool 2019
df_ManCity_19 = pd.read_csv('ManCity_2019.csv',header=None) # ManCity 2019
df_Chels_19 = pd.read_csv('Chelsea_2019.csv',header=None)   # Chelsea 2019

<div>
    <p>
        For the above databases I created by reading the created csv files I created a function that edits
        each database called <strong>cleanCsv</strong> that takes a single database as argument. The function switches the 
        rows and columns , sets the columns equal to the key values and removes the indexes.
    </p>
</div>

In [7]:
def cleanCsv(database):
    database = database.T                                # method to switch column rows and columns with respective values
    database = database.rename(columns=database.iloc[0]) # changing the first row of database to be equal to rows
    database = database.drop(labels=0, axis=0)           # removing all row names or indexes
    return database                                      # return updated database

<div>
    <p>
        Calling cleanCsv function on all databases:
    </p>
</div>

In [8]:
df_ManUtd_06 = cleanCsv(df_ManUtd_06)
df_Chels_06 = cleanCsv(df_Chels_06)
df_Liv_06 = cleanCsv(df_Liv_06)
df_Liv_19 = cleanCsv(df_Liv_19)
df_ManCity_19 = cleanCsv(df_ManCity_19)
df_Chels_19 = cleanCsv(df_Chels_19)

<div>
    <p>
        Because all databases now have the same attributes i can join them into one database by using the <strong>concat</strong> method
        from pandas library.
    </p>
</div>

In [9]:
mainDataFrame = pd.concat([df_ManUtd_06, df_Chels_06, df_Liv_06, df_Liv_19, df_ManCity_19 , df_Chels_19  ], axis=0)

<div>
    <p>
        This is how our dataframe now looks : 
    </p>
</div>

In [10]:
mainDataFrame

,wins,losses,goals,goals_conceded,clean_sheet,total_scoring_att,ontarget_scoring_att,att_pen_goal,hit_woodwork,total_pass,...,interception,total_clearance,own_goals,total_yel_card,total_red_card,total_offside,big_chance_created,effective_head_clearance,error_lead_to_goal,attempted_tackle_foul
1,28.0,5.0,83.0,27.0,16.0,698.0,256.0,5.0,21.0,18723.0,...,254.0,1222.0,1.0,60.0,1.0,80.0,NaN,NaN,NaN,NaN
1,24.0,3.0,64.0,24.0,22.0,636.0,216.0,3.0,14.0,16769.0,...,292.0,1206.0,1.0,62.0,4.0,127.0,NaN,NaN,NaN,NaN
1,20.0,10.0,57.0,27.0,20.0,668.0,214.0,6.0,15.0,17162.0,...,246.0,1115.0,NaN,44.0,NaN,120.0,NaN,NaN,NaN,NaN
1,32.0,3.0,85.0,33.0,15.0,591.0,231.0,5.0,17.0,23872.0,...,352.0,587.0,1.0,38.0,1.0,53.0,88.0,259.0,5.0,181.0
1,26.0,9.0,102.0,35.0,17.0,745.0,266.0,6.0,27.0,26329.0,...,352.0,476.0,1.0,60.0,4.0,69.0,108.0,241.0,8.0,202.0
1,20.0,12.0,69.0,54.0,9.0,625.0,223.0,7.0,14.0,23244.0,...,459.0,586.0,3.0,60.0,NaN,63.0,74.0,296.0,3.0,220.0


<div>
    <p>
        Below i'm creating a column named 'teams' , inserting it into the dataframe and setting the index equal to the column.
    </p>
</div>

In [11]:
mainDataFrame["teams"]=["ManUtd 2006", "Chelsea 2006" , "Liverpool 2006" , "Liverpool 2019", "ManCity 2019" , "Chelsea 2019"]
mainDataFrame.set_index('teams', inplace = True)

In [12]:
mainDataFrame

,wins,losses,goals,goals_conceded,clean_sheet,total_scoring_att,ontarget_scoring_att,att_pen_goal,hit_woodwork,total_pass,...,interception,total_clearance,own_goals,total_yel_card,total_red_card,total_offside,big_chance_created,effective_head_clearance,error_lead_to_goal,attempted_tackle_foul
teams,,,,,,,,,,,,,,,,,,,,,
ManUtd 2006,28.0,5.0,83.0,27.0,16.0,698.0,256.0,5.0,21.0,18723.0,...,254.0,1222.0,1.0,60.0,1.0,80.0,NaN,NaN,NaN,NaN
Chelsea 2006,24.0,3.0,64.0,24.0,22.0,636.0,216.0,3.0,14.0,16769.0,...,292.0,1206.0,1.0,62.0,4.0,127.0,NaN,NaN,NaN,NaN
Liverpool 2006,20.0,10.0,57.0,27.0,20.0,668.0,214.0,6.0,15.0,17162.0,...,246.0,1115.0,NaN,44.0,NaN,120.0,NaN,NaN,NaN,NaN
Liverpool 2019,32.0,3.0,85.0,33.0,15.0,591.0,231.0,5.0,17.0,23872.0,...,352.0,587.0,1.0,38.0,1.0,53.0,88.0,259.0,5.0,181.0
ManCity 2019,26.0,9.0,102.0,35.0,17.0,745.0,266.0,6.0,27.0,26329.0,...,352.0,476.0,1.0,60.0,4.0,69.0,108.0,241.0,8.0,202.0
Chelsea 2019,20.0,12.0,69.0,54.0,9.0,625.0,223.0,7.0,14.0,23244.0,...,459.0,586.0,3.0,60.0,NaN,63.0,74.0,296.0,3.0,220.0


<div>
    <p>
        Unfortunately my first flaw of the data i'm using is that there are more statistics taken by the premier league in the 2019
        season compared to the 2006 season. Therefore if you look to the right side of the database there are many 'NaN' values.
        I have decided to remove these columns since they could not be compared. This code follows:  
    </p>
</div>

In [13]:
mainDataFrame=mainDataFrame.dropna(axis=1)

<div>
    <p>
        Below i have renamed columns to have more readable and sensible names : 
    </p>
</div>

In [14]:
mainDataFrame.rename(columns={'wins':'Wins','losses':'Losses','goals':'Goals','goals_conceded': 'Goals Conceded','clean_sheet': 'Clean Sheets','total_scoring_att':'Shots','ontarget_scoring_att':'Shots on target','att_pen_goal':'Penalties scored','hit_woodwork':'Hit woodwork','total_pass':'Passes','interception':'Interceptions','total_clearance':'Total clearances'}, inplace=True)

C:\Users\chaan\anaconda3\lib\site-packages\pandas\core\frame.py:4441: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  return super().rename(


In [15]:
mainDataFrame.rename(columns={'total_cross':'Crosses','saves':'Saves','total_tackle':'Tackles','blocked_scoring_att':'Blockes shots','total_yel_card':'Yellow cards','total_offside':'Offsides'}, inplace=True)

In [16]:
mainDataFrame

,Wins,Losses,Goals,Goals Conceded,Clean Sheets,Shots,Shots on target,Penalties scored,Hit woodwork,Passes,...,Crosses,accurate_cross,Saves,Tackles,won_tackle,Blockes shots,Interceptions,Total clearances,Yellow cards,Offsides
teams,,,,,,,,,,,,,,,,,,,,,
ManUtd 2006,28.0,5.0,83.0,27.0,16.0,698.0,256.0,5.0,21.0,18723.0,...,918.0,226.0,2.0,890.0,677.0,146.0,254.0,1222.0,60.0,80.0
Chelsea 2006,24.0,3.0,64.0,24.0,22.0,636.0,216.0,3.0,14.0,16769.0,...,897.0,222.0,5.0,982.0,768.0,140.0,292.0,1206.0,62.0,127.0
Liverpool 2006,20.0,10.0,57.0,27.0,20.0,668.0,214.0,6.0,15.0,17162.0,...,1107.0,278.0,3.0,969.0,765.0,128.0,246.0,1115.0,44.0,120.0
Liverpool 2019,32.0,3.0,85.0,33.0,15.0,591.0,231.0,5.0,17.0,23872.0,...,896.0,180.0,76.0,550.0,325.0,164.0,352.0,587.0,38.0,53.0
ManCity 2019,26.0,9.0,102.0,35.0,17.0,745.0,266.0,6.0,27.0,26329.0,...,914.0,205.0,76.0,514.0,301.0,222.0,352.0,476.0,60.0,69.0
Chelsea 2019,20.0,12.0,69.0,54.0,9.0,625.0,223.0,7.0,14.0,23244.0,...,862.0,217.0,65.0,638.0,362.0,178.0,459.0,586.0,60.0,63.0


<div>
    <p>
        Below I have written a function i can use to insert columns into the dataframe . I only later realised that I was only 
        going to be using this function once and therefore is unnecassary , nevertheless I added the column 'Matches Played'
         which was constantly 38 for all teams
    </p>
</div>

In [17]:
def insertNewCol(columnName,index,a,b,c,x,y,z):
    mainDataFrame[columnName]=[a,b,c,x,y,z]
    some_column = mainDataFrame.pop(columnName)
    mainDataFrame.insert(index, columnName, some_column) 

In [18]:
insertNewCol("Matches Played",0,38,38,38,38,38,38)

<ipython-input-17-cbe477a11bc1>:2: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  mainDataFrame[columnName]=[a,b,c,x,y,z]


<div>
    <p>
        Below I have written a function called <strong>insertStatsPerGame</strong> that takes 2 parameters: columnName ,  newColName.
        This function creates a new column (newColName) that contains stats per game.
        Certain stats are divided by the total matches played (38) . The reason for this is because these stats i could not pull from the 
        webpage directly.
    </p>
</div>

In [19]:
def insertStatsPerGame(columnName , newColName):
    x = mainDataFrame[columnName].div(mainDataFrame['Matches Played'])    # divides by total matches to get stats per game
    x = x.round(2)                                                        # round to 2 decimal point
    mainDataFrame[newColName]=x                                           # assigne new column values

In [20]:
mainDataFrame = mainDataFrame.astype(int)

In [21]:
# inserting missing statistics into database
insertStatsPerGame('Passes','Passes per match')
insertStatsPerGame('Goals Conceded','Goals conceded per match')
insertStatsPerGame('Goals','Goals per game')

<div>
    <p>
        The following 4 cells calculates % of statistics that was unable to extract from the website and assigns new columns to
        these values.
    </p>
</div>

In [22]:
x = (mainDataFrame['accurate_pass'].div(mainDataFrame['Passes']))*100
x = x.round(2)
mainDataFrame["Passing Accuracy %"]=x  

In [23]:
y = (mainDataFrame['Shots on target'].div(mainDataFrame['Shots']))*100
y = y.round(2)
mainDataFrame["Shot Accuracy %"]=y  

In [24]:
z = (mainDataFrame['accurate_cross'].div(mainDataFrame['Crosses']))*100
z = z.round(2)
mainDataFrame["Cross accuracy %"]=z  

In [25]:
t = (mainDataFrame['won_tackle'].div(mainDataFrame['Tackles']))*100
t = t.round(2)
mainDataFrame["Tackle success %"]=t 

<div>
    <p>
        Working on the database as it is formulated now is easier but I believe the dataframe look more readable when flipping the rows and columns like this : 
    </p>
</div>

In [26]:
mainDataFrame.T

teams,ManUtd 2006,Chelsea 2006,Liverpool 2006,Liverpool 2019,ManCity 2019,Chelsea 2019
Matches Played,38.00,38.00,38.00,38.00,38.00,38.00
Wins,28.00,24.00,20.00,32.00,26.00,20.00
Losses,5.00,3.00,10.00,3.00,9.00,12.00
Goals,83.00,64.00,57.00,85.00,102.00,69.00
Goals Conceded,27.00,24.00,27.00,33.00,35.00,54.00
Clean Sheets,16.00,22.00,20.00,15.00,17.00,9.00
Shots,698.00,636.00,668.00,591.00,745.00,625.00
Shots on target,256.00,216.00,214.00,231.00,266.00,223.00
Penalties scored,5.00,3.00,6.00,5.00,6.00,7.00
Hit woodwork,21.00,14.00,15.00,17.00,27.00,14.00


<div>
    <p>
        Below i have removed 4 unnecassary stats that only had purpose for calculating % . 
    </p>
</div>

In [27]:
mainDataFrame = mainDataFrame.drop(['accurate_pass'],axis = 1) 
mainDataFrame = mainDataFrame.drop(['accurate_cross'],axis = 1) 
mainDataFrame = mainDataFrame.drop(['won_tackle'],axis = 1) 
mainDataFrame = mainDataFrame.drop(['Saves'],axis = 1) 

In [28]:
df1 = mainDataFrame.T

<div>
    <h3>
        3) Database final
    </h3>

</div>

In [29]:
mainDataFrame.T

teams,ManUtd 2006,Chelsea 2006,Liverpool 2006,Liverpool 2019,ManCity 2019,Chelsea 2019
Matches Played,38.00,38.00,38.00,38.00,38.00,38.00
Wins,28.00,24.00,20.00,32.00,26.00,20.00
Losses,5.00,3.00,10.00,3.00,9.00,12.00
Goals,83.00,64.00,57.00,85.00,102.00,69.00
Goals Conceded,27.00,24.00,27.00,33.00,35.00,54.00
Clean Sheets,16.00,22.00,20.00,15.00,17.00,9.00
Shots,698.00,636.00,668.00,591.00,745.00,625.00
Shots on target,256.00,216.00,214.00,231.00,266.00,223.00
Penalties scored,5.00,3.00,6.00,5.00,6.00,7.00
Hit woodwork,21.00,14.00,15.00,17.00,27.00,14.00


  <h3>
        4) Database Evaluation
    </h3>

In [30]:
total_2006 = mainDataFrame.iloc[0:3].sum()  # adding all elements of 2006
total_2019 = mainDataFrame.iloc[3:].sum()   # adding all elements from 2019

In [31]:
k =total_2019 - total_2006                   # calculating the difference of each stat 2019 vs 2006
k =k.round(2)                                # rounding values to 2 decimal places
l = ((total_2019-total_2006)/total_2006)*100 # calculating percentage 2019 is more than 2006
l =l.round(2)                                # rounding values to 2 decimal places 

<div>
    <p>
        Looping through all key value pairs in k and checking if stats are higher in 2006 or 2019. 
    </p>
</div>

In [32]:
total_2006_more = 0
total_2019_more= 0

In [33]:
for key,val in k.items():
    if val < 0:
        print('2006',val*-1 ,'More ', key , ' in 2006 than 2019'  )
        total_2006_more +=1
    if val > 0:
        print('2019',val ,'More ', key , ' in 2019 than 2006')
        total_2019_more +=1

2019 6.0 More  Wins  in 2019 than 2006
2019 6.0 More  Losses  in 2019 than 2006
2019 52.0 More  Goals  in 2019 than 2006
2019 44.0 More  Goals Conceded  in 2019 than 2006
2006 17.0 More  Clean Sheets  in 2006 than 2019
2006 41.0 More  Shots  in 2006 than 2019
2019 34.0 More  Shots on target  in 2019 than 2006
2019 4.0 More  Penalties scored  in 2019 than 2006
2019 8.0 More  Hit woodwork  in 2019 than 2006
2019 20791.0 More  Passes  in 2019 than 2006
2006 250.0 More  Crosses  in 2006 than 2019
2006 1139.0 More  Tackles  in 2006 than 2019
2019 150.0 More  Blockes shots  in 2019 than 2006
2019 371.0 More  Interceptions  in 2019 than 2006
2006 1894.0 More  Total clearances  in 2006 than 2019
2006 8.0 More  Yellow cards  in 2006 than 2019
2006 142.0 More  Offsides  in 2006 than 2019
2019 547.13 More  Passes per match  in 2019 than 2006
2019 1.16 More  Goals conceded per match  in 2019 than 2006
2019 1.38 More  Goals per game  in 2019 than 2006
2019 23.13 More  Passing Accuracy %  in 2019 th

In [34]:
# minus 2 because 2 of these stats are negative
total_2006_more - 2

7

In [35]:
# minus 3 because 3 of these stats are negative
total_2019_more - 3 

12

<div>
    <h4>
        Conclusion 2019 vs 2016 : Numerical
    </h4>
    <p>
        In 2006 there were a total of 9 statistics that were higher than 2019 , but 2 were negative stats namely yellow cards
        and offsides . In 2019 there were a total of 15 statistics that were higher then 2009 , but 3 of these were negative statistics.
        Therefore 2019 has 5 more positive statistics than 2006 in the top 3 of the premier league.
    </p>
</div>

In [36]:
# assigning percentage stats 2019 vs 2006 to databases

In [37]:
db_2006 = {}
db_2019 = {}

for key,val in l.items():
    if val < -25:               # only assigning percentages to database if higher than 25 % difference
        db_2006[key ]=val
    elif val > 25:
        db_2019[key]=val        # only assigning percentages to database if higher than 25 % difference

In [38]:
for val in db_2006:
    db_2006[val] *=-1

In [39]:
db_2006

{'Clean Sheets': 29.31,
 'Tackles': 40.09,
 'Total clearances': 53.46,
 'Offsides': 43.43,
 'Tackle success %': 25.23}

In [40]:
db_2019

{'Losses': 33.33,
 'Goals': 25.49,
 'Goals Conceded': 56.41,
 'Penalties scored': 28.57,
 'Passes': 39.49,
 'Blockes shots': 36.23,
 'Interceptions': 46.84,
 'Passes per match': 39.49,
 'Goals conceded per match': 56.59,
 'Goals per game': 25.75}

<div>
    <h4>
        Conclusion 2019 vs 2016 : Percentage
    </h4>
    <p>
        The above 2 lists indicates the percentage above the average of each stat , they only contain percentages higher then
        25 % since their relevence are higher. 
    </p> 
    <p>
        In 2006 there was 5 high percentages where one percentage is a bad one namely 
        offsides. That being said it is interesting to see that in 2006 there were 40 % more tackles , 30 % more clean sheets 
        , 53 % more total clearances , and 25 % more tackle success rate. It is clear that in 2006 teams were much more defensive
        therefore maybe the reason for more offsides was a better defence.
    </p>
    <p>
        In 2019 there was 10 high percentages where 3 were bad stats namely : goals conceded , goals conceded per game and losses.
        In 2019 there were 56 % more goals conceded than the average , clearly the defences of 2006 were much more capable and 
        impactful. That being said there are 40 % more passes per match , 46 % more interceptions and 25 % more goals , this 
        could indicate that football has changed and become more free flowing and may conclude that there were better midfielders
        and forward thinking players in 2019
    </p>
</div>

 <p>
         Creating empty dictionaries to use to add and compare max values of each team and collectively each 
    </p>

In [41]:
manchesterUnited_2006={}
Chelsea_2006 = {}
Liverpool_2006 = {}
Liverpool_2019 = {}
ManCity_2019 = {}
Chelsea_2019 = {}

In [42]:
# ASSIGNING KEY VALUE PAIRS WHERE THE STATS ARE THE HIGHEST OF ALL SEASONS
for key , val in mainDataFrame.items():
    if(val.idxmax()=="ManUtd 2006"):
        manchesterUnited_2006[key]=[val.max()]
        
    if(val.idxmax()=="Chelsea 2006"):
        Chelsea_2006[key]=[val.max()]
        
    if(val.idxmax()=="Liverpool 2006"):
        Liverpool_2006[key]=[val.max()]
        
    if(val.idxmax()=="Liverpool 2019"):
        Liverpool_2019[key]=[val.max()]
        
    if(val.idxmax()=="ManCity 2019"):
        ManCity_2019[key]=[val.max()]
        
    if(val.idxmax()=="Chelsea 2019"):
        Chelsea_2019[key]=[val.max()]

   <p>
        Create new database for key stats .
    </p>

In [43]:

manchesterUnited_2006_df = pd.DataFrame(manchesterUnited_2006)
Chelsea_2006_df = pd.DataFrame(Chelsea_2006)
Liverpool_2006_df = pd.DataFrame(Liverpool_2006)
Liverpool_2019_df = pd.DataFrame(Liverpool_2019)
ManCity_2019_df = pd.DataFrame(ManCity_2019)
Chelsea_2019_df = pd.DataFrame(Chelsea_2019)
keyStats_Data = pd.concat([manchesterUnited_2006_df, Chelsea_2006_df, Liverpool_2006_df, Liverpool_2019_df, ManCity_2019_df , Chelsea_2019_df  ], axis=0)

In [44]:
keyStats_Data = keyStats_Data.fillna('-')

In [45]:
keyStats_Data["teams"]=["ManUtd 2006", "Chelsea 2006" , "Liverpool 2006" , "Liverpool 2019", "ManCity 2019" , "Chelsea 2019"]
keyStats_Data.set_index('teams', inplace = True)

In [46]:
keyStats_Data = keyStats_Data.T

 <p>
        Below are all the top stats comparing every team.
    </p>

In [47]:
keyStats_Data

teams,ManUtd 2006,Chelsea 2006,Liverpool 2006,Liverpool 2019,ManCity 2019,Chelsea 2019
Matches Played,38.0,-,-,-,-,-
Total clearances,1222.0,-,-,-,-,-
Clean Sheets,-,22.0,-,-,-,-
Tackles,-,982.0,-,-,-,-
Yellow cards,-,62.0,-,-,-,-
Offsides,-,127.0,-,-,-,-
Crosses,-,-,1107.0,-,-,-
Tackle success %,-,-,78.95,-,-,-
Wins,-,-,-,32.0,-,-
Shot Accuracy %,-,-,-,39.09,-,-


<div>
    <h4>
        Conclusion Teams and Seasons : KeyStats
    </h4>
    <p>
        Comparing by seasons: There were only 7 statistics in 2006 that were the highest of all teams , all except crosses are defensive stats.
        In 2019 there were 17 of the highest stats of all teams with almost all of them being fovcused on attacking football and either
        scoring or conceding goals.
    </p> 
    <p>
        The team with the most key stats is Manchester city (9), with all of their stats being positive and very impressive one would
        think they were the winners of their year ... Unfortunately as we see the most important keystat of all is wins! Liverpool
        being champions in 2019 and Manchester United being champions in 2006 both have equally lacking keystats.
    </p>
    
</div>

 <p>
        Below i will be looking at the percentage of above average stats of each team.
        The creation of databases and calculations of averages follow:
    </p>

In [48]:
columnSum = mainDataFrame.sum(axis=0)

In [49]:
columnSum_avg = columnSum/6

In [50]:
columnSum_avg = columnSum_avg.round(2)

In [51]:
manUtd_2006_row_db = {}
chelsea_2006_row_db =  {}
liverpool_2006_row_db =  {}
liverpool_2019_row_db = {}
manCity_2019_row_db =  {}
chelsea_2019_row_db = {}

In [52]:
manUtd_2006_row = mainDataFrame.iloc[0]
chelsea_2006_row = mainDataFrame.iloc[1]
liverpool_2006_row = mainDataFrame.iloc[2]
liverpool_2019_row = mainDataFrame.iloc[3]
manCity_2019_row = mainDataFrame.iloc[4]
chelsea_2019_row = mainDataFrame.iloc[5]

In [53]:
for i in range(0,len(columnSum_avg)):
    if manUtd_2006_row[i] > columnSum_avg[i]:
        roundval1=((manUtd_2006_row[i]/columnSum_avg[i])-1)*100
        roundval1 = roundval1.round(2)
        manUtd_2006_row_db[manUtd_2006_row.index[i]] = roundval1
        
      
        
    if chelsea_2006_row[i] > columnSum_avg[i]:
        roundval2=((chelsea_2006_row[i]/columnSum_avg[i])-1)*100
        roundval2 = roundval2.round(2)
        chelsea_2006_row_db[chelsea_2006_row.index[i]] = roundval2
        
    
        
    if liverpool_2006_row[i] > columnSum_avg[i]:
        roundval3=((liverpool_2006_row[i]/columnSum_avg[i])-1)*100
        roundval3 = roundval3.round(2)
        liverpool_2006_row_db[liverpool_2006_row.index[i]] = roundval3
        
     
    
    if liverpool_2019_row[i] > columnSum_avg[i]:
        roundval4=((liverpool_2019_row[i]/columnSum_avg[i])-1)*100
        roundval4 = roundval4.round(2)
        liverpool_2019_row_db[liverpool_2019_row.index[i]] = roundval4
        
  
    
    if manCity_2019_row[i] > columnSum_avg[i]:
        roundval5=((manCity_2019_row[i]/columnSum_avg[i])-1)*100
        roundval5 = roundval5.round(2)
        manCity_2019_row_db[manCity_2019_row.index[i]] = roundval5
        

    
    if chelsea_2019_row[i] > columnSum_avg[i]:
        roundval6=((chelsea_2019_row[i]/columnSum_avg[i])-1)*100
        roundval6 = roundval6.round(2)
        chelsea_2019_row_db[chelsea_2019_row.index[i]] = roundval6
        
        
    

In [54]:
manUtd_2006_row_df = pd.DataFrame(manUtd_2006_row_db,index=['Manchester United 2006'])
chelsea_2006_row_df = pd.DataFrame(chelsea_2006_row_db ,index=['Chelsea 2006'])
liverpool_2006_row_df = pd.DataFrame( liverpool_2006_row_db,index=['Liverpool 2006'])
liverpool_2019_row_df = pd.DataFrame( liverpool_2019_row_db,index=['Liverpool 2019'])
manCity_2019_row_df = pd.DataFrame( manCity_2019_row_db,index=['Manchester City 2019'])
chelsea_2019_row_df = pd.DataFrame( chelsea_2019_row_db,index=['Chelsea 2019'])

In [55]:
Percentage_AboveAvg_Data = pd.concat([manUtd_2006_row_df, chelsea_2006_row_df, liverpool_2006_row_df, liverpool_2019_row_df, manCity_2019_row_df , chelsea_2019_row_df  ], axis=0)

In [56]:
Percentage_AboveAvg_Data = Percentage_AboveAvg_Data .fillna('-')
Percentage_AboveAvg_Data.T

,Manchester United 2006,Chelsea 2006,Liverpool 2006,Liverpool 2019,Manchester City 2019,Chelsea 2019
Wins,12.0,-,-,28.0,4.0,-
Goals,8.26,-,-,10.86,33.04,-
Shots,5.68,-,1.14,-,12.79,-
Shots on target,9.25,-,-,-,13.52,-
Hit woodwork,16.67,-,-,-,50.0,-
Tackles,17.54,29.69,27.98,-,-,-
Total clearances,41.22,39.37,28.85,-,-,-
Yellow cards,11.11,14.81,-,-,11.11,11.11
Goals per game,7.92,-,-,10.89,32.67,-
Shot Accuracy %,3.24,-,-,10.02,0.48,0.42


<div>
    <h4>
        Conclusion Teams and Seasons : Percentage average data
    </h4>
    <p>
         Looking at the teams in 2006 we can conclude that once again these teams' defensive capabilities 
        are much higher then the average. It is clear to see that chelsea and liverpool ,whom did not win the league that season 
        , almost only have above average defensive stats. Manchester united , the champions of 2006 are the only team with 
        attacking statistics above average in this season. In 2019 we can clearly see the pattern of above average attacking minded football.
        Liverpool and Chelsea in 2019 both only have attacking minded statistics above average. Manchester City on the otherhand clearly
        dominates almost every statistic having only missed out on 6 out of 24 stats above average. They scored 33% more then the average 
        , hit the woodwork 50% more than the average and scored 32 % more goals per game. With chelsea in 2019 losing 70% more than the average
        and conceded 62 % more goals then the average team in this dataframe.
    </p> 
  
    
</div>

<div>
    <h3>
        Conclusion 
    </h3>
    <p>
        Clearly there was a pattern of defensive minded football being played in 2006 , the reason for this might be that the video
        assistant refferee was added to football in 2019. Therefore more chances were taken regarding tackles in 2006 but that being 
        said the tacke success percentage was also much higher. I can conclude that the teams from 2006 had better defences . As to why this 
        is so is a possible research criteria for continueing this project .
    </p> 
    <p>
        In 2019 we see much more goals , goals conceded and midfield play like passes and interceptions. this concludes that the attacking 
        play of football teams in the modern age is much more lethal and free flowing. Maybe higher quality players have been added due to 
        more lucritive teams and maybe the refereeing is much more strict and favours the attacking team. I conclude that teams from 2019 
        have much better attacking capabilities. To understand why this is the way it is , is another possible future research criteria.
    </p>
    <p>
        The Best Team : Looking at statistics it is clear to see that Manchester City dominates most fields , especially attacking play.
        Unfortunately the team with the best stats did not win in 2019... this is not the same for 2006. Manchester United dominated statistics
        compared to their competitors in 2006 and do a decent job at competing with the newer teams. Manchester United won the league in 2006.
        Liverpool in 2019 won the league , but did not dominate stats compared to their competitors past or present but performed solidly 
        at a few of the comparison dataframes. Unfortunately i'm a Chelsea fan and like i mentionded earlier football pundits , players and 
        fans are bias ... so chelsea was the best team for me !
    </p>
    
</div>

<div>
    <h3>
        Ethics :
    </h3>
    <p>
        Reading from the following webpage terms and conditions : <a href="https://www.premierleague.com/terms-and-conditions">https://www.premierleague.com/terms-and-conditions</a>
        Under intelectual property rights it clearly states that we can download and print material from this website , you can forward material from the website to other people 
        for their private and personal use as long as you use the link of the website. No dangerous or harmful implications will come 
        from using this data since its only numerical stats of the past. The analysis does have potential to create new forms of intellectual 
        property to be used by football pundits and fans , a possible future project would be making all code reusable so that any season could
        be compared and analysed.
    </p>
  
</div>